# Tutorial para trabajar en Paralelo en Julia 1.0.x

Haste un buen tutorial para trabajar en paralelo, especialmente las funciones vergonzosamente paralelas.

In [1]:
# A partir de ahora, Julia tiene las funciones básicas de paralelo en Distributed

using Distributed, SharedArrays

addprocs(4) #por ejemplo, esta esta en Distributed

4-element Array{Int64,1}:
 2
 3
 4
 5

In [2]:
A=SharedArray{Float64}(8,8,100)
B=SharedArray{Float64}(8,8,100);

 La forma de abajo rellena A con valores aleatoreos. Como se puede ver, es más rapido que la forma serial en un sólo procesador, sin embargo no es MUCHO mas rapida, dado que no estamos explotando la Vergonzosa Paralelización

In [6]:
@time A=@sync randn(64,64,8000);  

  0.344100 seconds (13 allocations: 250.000 MiB, 4.26% gc time)


In [7]:
@time C=randn(64,64,8000);

  0.397400 seconds (10 allocations: 250.000 MiB, 17.13% gc time)


In [15]:
function enteroarangos(nn::Int)
    result=[]
    np=nprocs()
    if np==1
        result=[1:nn]
    else
          bb=div(nn,np)
        for j=1:(np-1)
           
            aux=((j-1)*bb+1):j*bb
            push!(result, aux)
        end
        aux=(np-1)*bb:nn
        push!(result,aux)
    end
    return result
end

enteroarangos (generic function with 1 method)

Esto parece estar aun peor:

In [41]:
rangos=enteroarangos(8000);
@time @sync begin
for p in procs()
        r=rangos[p]
        rl=length(r)
    @async A[:,:,r]=remotecall_fetch(randn, p, 64,64,rl)
        print(p, " ")
    end
end

1 2 3 4 5   0.457217 seconds (11.30 k allocations: 250.561 MiB, 29.91% gc time)


In [43]:
A[:,:,5000]

64×64 Array{Float64,2}:
 -0.699769    -0.618232   -1.17597    …   0.11664     0.802312   -0.871803 
 -0.971473    -0.0669316   0.242634      -0.832293   -0.250825    1.64553  
  1.87582      1.71093    -0.778121       0.658627   -0.0977677   0.222937 
 -0.352145     1.39128    -0.303881       0.639688    0.649276   -0.609182 
 -0.234354     0.53354     0.820147       0.566679   -1.05966     0.457698 
 -0.465027     0.160488   -0.428067   …   0.68727     1.48003    -0.232314 
  3.01104      0.817164    0.758898      -1.28944    -0.993045    1.1996   
 -1.61833      2.23881     1.18639        0.635698   -0.69447     0.193327 
  1.51609     -0.811188    1.71791        1.33464     1.45405     1.60332  
 -2.03454      1.96529     1.35733        0.100102   -0.483451    0.815067 
 -2.42917      0.118585   -2.16413    …  -0.528577   -0.680802    0.312458 
  0.00621477   0.262157   -0.89777       -0.430316   -0.365693   -0.532293 
 -1.48747      0.209509    0.835445      -1.09483    -0.354053  

In [ ]:
s=@spawnat 2 2 .+ fetch(a)

In [ ]:
fetch(s)

In [ ]:
?Future

In [ ]:
M = Matrix{Float64}[rand(1000,1000) for i = 1:10];

pmap(svdvals, M)

In [ ]:
using LinearAlgebra

In [ ]:
> M = Matrix{Float64}[rand(1000,1000) for i = 1:10];

julia> pmap(svdvals, M);

In [ ]:

@everywhere fdelta(datos)=filtrobanda(rangofreqs,datos, 0.0001, 0.0035)
@everywhere ftheta(datos)=filtrobanda(rangofreqs, datos, 0.0035, 0.007022)
@everywhere falpha(datos)=filtrobanda(rangofreqs, datos, 0.007022, 0.014044)
@everywhere fbeta(datos)=filtrobanda(rangofreqs, datos, 0.014044, 0.035120)
@everywhere fgamma(datos)=filtrobanda(rangofreqs, datos, 0.035120, 0.100)
@everywhere fspr(datos)=filtrobanda(rangofreqs, datos, 0.100, 0.250)
@everywhere fuspr(datos)=filtrobanda(rangofreqs, datos, 0.250, 0.600)


@everywhere function podpromediobanda(datos::Array, cventana=350, banda="gamma")
    
nmax=size(datos)[3]    
cmax=nmax-cventana
    
result=zeros(64,64,cmax)

dictfiltros=Dict("alpha"=>falpha, "beta"=>fbeta, 
        "gamma"=>fgamma, "theta"=>ftheta, 
        "delta"=>fdelta, "spr"=>fspr, "uspr"=>fuspr )          

filt=dictfiltros[banda]
    
for j=1:64, k=1:64
   wtaux=wfft(lfp[j,k,:], cventana)
   for t=1:cmax
        auxfilt=filt(wtaux[t,:])
        result[j,k,t]=dot(auxfilt,auxfilt)
    end
return resul

end

end

In [ ]:
A=randn(8,8,100) 
@everywhere aux(datos)=podpromediobanda(datos, 4, "gamma")
B=pmap(aux, A)